# Medical Text Clustering

The MTSamples Dataset has medical transcriptions from several conditions and consultations. It also provides a medical speciality and keywords extracted manually from the transcription. The information however, is not very specific. For example, all of the `surgery` consultations are under the same `medical_speciality` field. Nonetheless, "surgery" is a very broad term, and it can go from minor molar procedures to life-or-death cardiovascular operations. 

The task is to run a clustering algorithm that is able to provide sub-groups within the surgery speciality. The end-goal is to have sub-topics clearly identified under the surgery category. 

In [ ]:
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
# load from our own module
# Follow installation in the Readme beforehand!
from medical_nlp import utils

In [ ]:
# Set up

# Load the model!
# For vectors, at least a size MD model is needed!
nlp = spacy.load("en_core_sci_md")

txt_col : str = "transcription"
data_path : str = "../data/mtsamples.csv"

# You can play with the params and see different results!
MODEL_PARAMS: dict = {
    "eps": 0.25,
    "min_samples": 12
}

## Quick Data Exploration

In [ ]:
df = pd.read_csv(data_path)
# drop nas from the text column
df = df[df[txt_col].notna()]


In [ ]:
# Top ten specialities
df.medical_specialty.value_counts()[:10].plot(kind="bar", title="Top Ten medical specialities in the data");

In [ ]:
df.head()

#### Filtering for only Surgery Speciality

In [ ]:
# Get only Surgery topics 
df = df[df["medical_specialty"] == " Surgery"].reset_index(drop=True)
# process the keywords as it will be easier to handle as a list of lists
df["processed_kw"] = df.keywords.apply(lambda row: utils.process_keywords(row))

## NLP

To make our clusters, we need to first have our text inputs in a vectorial representation. This will allow us to use traditional unsupervised machine learning methods to get our clusters.
Transforming text to clusters can be done in multiple ways (from basic frequency counts to the state-of-the-art deep learning Transformer models), however for this tutorial we will use our SciSpacy model to get the vectors we need. 

SciSpacy (and Spacy in general) makes it really simple as they have already developed execellent methods to vectorize and get text similarities in their core library.

In [ ]:
# Get the vector representation of the transcription
# Takes around 5 minutes!
df["vectors"] = df[txt_col].apply(
    lambda sent : utils.get_vectors(sent, nlp)
)

## Build the Clustering Model 

We are going to use the DBSCAN unsupervised algorithm to build our clusters. This algorithm is particulary useful in finding non-linear relationships, based on densities. It does not require the user to "pre-define" a number of clusters (contrary to K-Means for example) and it works well with high dimmensions (such as word embeddings). 

More info: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

Advantages:

- You don't need to define the number of K.
- Robust to Oultiers, since its definition "labels" them as such if they do not have a considerable number of neighbors (low density)
- Excellent algorithm on arbitrary distributions

In [ ]:
# Create DBScan model
# setting in the format DBScan requires
embeddings = utils.format_embeddings_for_clustering(df.vectors.values)  
# Performing DBSCAN clustering
dbscan = DBSCAN(
    **MODEL_PARAMS
)
# Get the clusters. Remember, -1 == outlier (Noise)!
labels = dbscan.fit_predict(embeddings)
# Add the labels back to the DF
df["cluster"] = labels

In [ ]:
# Visualizing the clustering result
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Our embeddings have 200 dimmensions, and of course, we can't plot in 200D. 
# The best we can do is 3D, so we choose a random set of dimmensions.
d1 = 10
d2 = 20
d3 = 30
# Plot points belonging to cluster -1 with reduced alpha and a different marker
ax.scatter(
    embeddings[dbscan.labels_ == -1, d1], 
    embeddings[dbscan.labels_ == -1, d2], 
    embeddings[dbscan.labels_ == -1, d3], 
    c='black', marker='x', alpha=0.2, label='Noise'
)

# Plot points belonging to clusters other than -1
for label in set(dbscan.labels_) - {-1}:
    ax.scatter(
        embeddings[dbscan.labels_ == label, d1], 
        embeddings[dbscan.labels_ == label, d2], 
        embeddings[dbscan.labels_ == label, d3], 
        label=f'Cluster {label}'
)

ax.set_title('DBSCAN Clustering viz (3D)')
ax.set_xlabel('Dim1')
ax.set_ylabel('Dim2')
ax.set_zlabel('Dim3')
ax.legend()
plt.show()

## Evaluate Clusters

Now that we have our clusters, we need to evaluate that they have been succesfully grouped together by their semantic similarity. In other words, we expect that similar topics would be clustered together.
Let's give it a look

In [ ]:
# build a dictionary that will store most frequent terms
# per label
TOP_N : int = 10  # Top 10 terms
freq_dict: dict = {}

for label_ in df.cluster.unique():
    if label_ != -1:  # skip noise
        cluster_sample = df.query("cluster == @label_")
        # skip "surgery" (present in all) and empty keywords.
        exclude = ["surgery", ""]
        # count word freq
        counts = utils.get_most_common(
            cluster_sample.processed_kw.values,
            top_n=TOP_N,
            exclude=exclude
        )
        # assign counter to dictionary.
        freq_dict[label_] = counts


In [ ]:
# Let's give it a look
for key in freq_dict:
    print(f"Cluster {key}")
    print(freq_dict[key])
    print()

We now have our clusters. What can you see?

- **Cluster 0** has to do with bone injuries, like broken feet, knee surgeries, cervical issues. 
- **Cluster 1** is clearly related to eye issues. Cataract and intraocular surgeries.
- **Cluster 2** is related to genitals and reproductory organs mainly.
- **Cluster 3** treats heart and cardivascular operations. 
- **Cluster 4** Mainly covers surgeries around the stomach and digestive system. 

The algorithm has "learnt" relevant patterns and has created actionable clusters! 

End of file